## data updates for the helioforecast.space website

Main author: C. Möstl, Austrian Space Weather Office, GeoSphere Austria

https://github.com/cmoestl/heliocats

uses environment 'envs/env_helio4.yml'

check:
- switches on
- plotting set to 'Agg'
- .py file is current (execute first cell)

In [10]:
# https://github.com/cmoestl/heliocats  data_update_web.py

# for updating data every day on the servers

# MIT LICENSE
# Copyright 2020-2023, Christian Moestl 
# Permission is hereby granted, free of charge, to any person obtaining a copy of this 
# software and associated documentation files (the "Software"), to deal in the Software
# without restriction, including without limitation the rights to use, copy, modify, 
# merge, publish, distribute, sublicense, and/or sell copies of the Software, and to 
# permit persons to whom the Software is furnished to do so, subject to the following 
# conditions:
# The above copyright notice and this permission notice shall be included in all copies 
# or substantial portions of the Software.
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, 
# INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A
# PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT 
# HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF 
# CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE 
# OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

import pickle
import importlib
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.dates as mdates
import sys
import numpy as np
import datetime
import scipy.signal
import urllib
import json
import os   
import time
import h5py
import pytz
import copy
import cdflib

#import 
from heliocats import data as hd
importlib.reload(hd) #reload again while debugging

from heliocats import plot as hp
importlib.reload(hp) #reload again while debugging



################################################ CHECK  ##############################################
matplotlib.use('Agg')   #for server
#%matplotlib inline     #for mac

import os
os.system('jupyter nbconvert --to script data_update_web.ipynb')   


#switches
get_omni=1
get_noaa=1
get_wind=1
get_stereoa=1

####################################################################################################################


#test execution times
t0all = time.time()


[NbConvertApp] Converting notebook data_update_web.ipynb to script
[NbConvertApp] Writing 21190 bytes to data_update_web.py


### Configure paths


In [11]:
from config import data_path


print(' ')
print('------ PATHS ')

print(data_path)

plot_path=data_path+'plots/'
position_path=data_path+'plots_positions/'
sun_path=data_path+'plots_sun/'

print(plot_path)
print(position_path)

from config import noaa_path
print(noaa_path)

from config import wind_path
print(wind_path)

from config import stereoa_path
print(stereoa_path)

from config import data_path_ml
print(data_path_ml)


########### make directories first time
if os.path.isdir(plot_path) == False: os.mkdir(plot_path)
if os.path.isdir(plot_path+'omni2') == False: os.mkdir(plot_path+'omni2')
if os.path.isdir(plot_path+'wind') == False: os.mkdir(plot_path+'wind')
if os.path.isdir(plot_path+'stereoa') == False: os.mkdir(plot_path+'stereoa')
if os.path.isdir(plot_path+'combined') == False: os.mkdir(plot_path+'combined')


if os.path.isdir(position_path) == False: os.mkdir(position_path)
if os.path.isdir(sun_path) == False: os.mkdir(sun_path)
if os.path.isdir(noaa_path) == False: os.mkdir(noaa_path)
if os.path.isdir(data_path_ml) == False: os.mkdir(data_path_ml)

 
------ PATHS 
/Users/chris/python/data/insitu_python/
/Users/chris/python/data/insitu_python/plots/
/Users/chris/python/data/insitu_python/plots_positions/
/Users/chris/python/ops/data/noaa/
/Users/chris/python/data/wind/
/Users/chris/python/data/stereoa/
/Users/chris/python/data/insitu_python/ML_struct_arrays/


## Development cells

### positions and SDO plot

In [12]:
print(' ')
print('------ POSITIONS ')

# spacecraft positions image
hp.plot_positions(datetime.datetime.utcnow(),position_path, 'HEEQ',now=True)

print(' ')
print('------ SDO realtime images ')


# get current SDO images 
hd.get_sdo_realtime_image(sun_path)

 
------ POSITIONS 
saved as  /Users/chris/python/data/insitu_python/plots_positions/positions_2023_Jul_13.png
saved as  /Users/chris/python/data/insitu_python/plots_positions/positions_now.png
 
------ SDO realtime images 
saved  /Users/chris/python/data/insitu_python/plots_sun/latest_1024_0193.jpg
saved  /Users/chris/python/data/insitu_python/plots_sun/latest_1024_HMIB.jpg


### OMNI2 data


In [4]:
print(' ')
print('------ OMNI2 ')


# OMNI2
fileomni="omni_1963_now.p"
#this function downloads and saves the the omni2 data
if get_omni: 
    hd.save_omni_data(data_path,fileomni)
else:
    print('OMNI data NOT downloaded, turn on switch')
[o,ho]=pickle.load(open(data_path+fileomni, "rb" ) )  

start=datetime.datetime.utcnow() - datetime.timedelta(days=365)
end=datetime.datetime.utcnow() 
hp.plot_insitu_update(o, start, end,'OMNI2',plot_path+'omni2/',now=True)

 
------ OMNI2 
OMNI data NOT downloaded, turn on switch
saved as  /Users/chris/python/data/insitu_python/plots/omni2/OMNI2_2022_Jul_13_2023_Jul_13.png
saved as  /Users/chris/python/data/insitu_python/plots/omni2/OMNI2_2022_Jul_13_2023_Jul_13.pdf
saved as  /Users/chris/python/data/insitu_python/plots/omni2/OMNI2_now.png
saved as  /Users/chris/python/data/insitu_python/plots/omni2/OMNI2_now.pdf


### NOAA real time solar wind and Dst

In [5]:
print(' ')
print('------ NOAA real time solar wind data ')

#test execution times
t0 = time.time()


if get_noaa > 0:
    print('download NOAA real time solar wind plasma and mag and dst')
    datestr=str(datetime.datetime.utcnow().strftime("%Y-%m-%d"))
    print(datestr+' UTC')

    plasma='http://services.swpc.noaa.gov/products/solar-wind/plasma-7-day.json'
    mag='http://services.swpc.noaa.gov/products/solar-wind/mag-7-day.json'
    dst='https://services.swpc.noaa.gov/products/kyoto-dst.json'

    try: 
        urllib.request.urlretrieve(plasma, noaa_path+'plasma/plasma-7-day_'+datestr+'.json')
        print(noaa_path+'plasma/plasma-7-day_'+datestr+'.json')
    except urllib.error.URLError as e:
        print(' ', plasma,' ',e.reason)

    try: 
        urllib.request.urlretrieve(mag, noaa_path+'mag/mag-7-day_'+datestr+'.json')
        print(noaa_path+'mag/mag-7-day_'+datestr+'.json')
    except urllib.error.URLError as e:
        print(' ', mag,' ',e.reason)
        

    try: 
        urllib.request.urlretrieve(dst, noaa_path+'dst/dst-7-day_'+datestr+'.json')
        print(noaa_path+'dst/dst-7-day_'+datestr+'.json')
    except urllib.error.URLError as e:
        print(' ', mag,' ',e.reason)
 
    print('NOAA download complete')

else:
    print('NOAA data NOT downloaded, turn on switch')  

  #import 
from heliocats import data as hd
importlib.reload(hd) #reload again while debugging

from heliocats import plot as hp
importlib.reload(hp) #reload again while debugging
    
        
######## SAVE NOAA DATA AS PICKLE


save_noaa=1
filenoaa='noaa_rtsw_last_300files_now.p'
# last parameter gives a cutoff, so only the latest N files are taken for the NOAA data pickle file
if save_noaa > 0: hd.save_noaa_rtsw_data(data_path,noaa_path,filenoaa,300)

[noaa,hnoaa]=pickle.load(open(data_path+filenoaa, "rb" ) ) 
 
#plot
start=datetime.datetime.utcnow() - datetime.timedelta(days=55)
end=datetime.datetime.utcnow() 
hp.plot_insitu_update_noaa_rtsw(noaa, start, end,'NOAA_RTSW',plot_path+'noaa/',now2=True)


t1 = time.time()

print()
print('NOAA download, save as pickle and plotting takes', np.round(t1-t0,2), 'seconds')


    

 
------ NOAA real time solar wind data 
NOAA data NOT downloaded, turn on switch
 
convert NOAA real time solar wind archive to pickle file
directories for the json data
/Users/chris/python/ops/data/noaa/mag/
Sorted file list to be read with cutoff  300  files. 
['mag-7-day_2023-06-06T23h.json' 'mag-7-day_2023-06-07T05h.json'
 'mag-7-day_2023-06-07T11h.json' 'mag-7-day_2023-06-07T17h.json'
 'mag-7-day_2023-06-07T23h.json' 'mag-7-day_2023-06-08T05h.json'
 'mag-7-day_2023-06-08T11h.json' 'mag-7-day_2023-06-08T17h.json'
 'mag-7-day_2023-06-08T23h.json' 'mag-7-day_2023-06-09T05h.json'
 'mag-7-day_2023-06-09T11h.json' 'mag-7-day_2023-06-09T17h.json'
 'mag-7-day_2023-06-09T23h.json' 'mag-7-day_2023-06-10T05h.json'
 'mag-7-day_2023-06-10T11h.json' 'mag-7-day_2023-06-10T17h.json'
 'mag-7-day_2023-06-10T23h.json' 'mag-7-day_2023-06-11T05h.json'
 'mag-7-day_2023-06-11T11h.json' 'mag-7-day_2023-06-11T17h.json'
 'mag-7-day_2023-06-11T23h.json' 'mag-7-day_2023-06-12T05h.json'
 'mag-7-day_2023-06-1

### Wind data

In [6]:

print(' ')
print('------ download Wind data ')


#download data for current year only    
if get_wind > 0:
    hd.wind_download_ascii(start_year=datetime.datetime.utcnow().year, wind_path=wind_path) 
else:
    print('Wind data NOT downloaded, turn on switch')  

    
    
    
#filewin="wind_2018_now_heeq.p" 
#start=datetime.datetime(2022, 12, 1)
#start=datetime.datetime(2022, 12, 1)
#end=datetime.datetime.utcnow()

#hd.save_wind_data_ascii(data_path,filewin,start,end,coord='HEEQ')
#[win,winh]=pickle.load(open(data_path+filewin, "rb"))
#start=win.time[-1]-datetime.timedelta(days=365)
#end=datetime.datetime.utcnow()         
#hp.plot_insitu_update(win, start, end,'Wind',plot_path+'wind/',now=True)



 
------ download Wind data 
Wind data NOT downloaded, turn on switch


### STEREO-A beacon data

In [7]:
print(' ')
print('------ download STEREO-A beacon data ')
print(' ')

#test execution times
t0 = time.time()


if get_stereoa > 0:
    hd.stereoa_download_beacon(start_year=datetime.datetime.utcnow().year,start_month=datetime.datetime.utcnow().month,start_day=datetime.datetime.utcnow().day-5,stereoa_path=stereoa_path)   

print(' ')
print('------ process STEREO-A beacon data to pickle') 
   
#define filename
file_sta_beacon='stereoa_beacon_last_100days_now.p'   

#save pickle file
hd.save_stereoa_beacon_data(stereoa_path,file_sta_beacon,datetime.datetime.utcnow()-datetime.timedelta(days=100),datetime.datetime.utcnow(),coord='RTN' )   
    
#load pickle    
[sta,hsta]=pickle.load(open(data_path+file_sta_beacon, "rb" ) )  

#plot
start=datetime.datetime.utcnow() - datetime.timedelta(days=55)
end=datetime.datetime.utcnow() 
hp.plot_insitu_update_stereoa_beacon(sta, start, end,'STEREO-A_beacon',plot_path+'stereoa/',now2=True)
    
    
t1 = time.time()

print()
print('STEREO-A beacon downloading current month, save as pickle last 30 days and plotting takes', np.round(t1-t0,2), 'seconds')
    

 
------ download STEREO-A beacon data 
 
 
------ process STEREO-A beacon data to pickle
2023-04-04 11:55:00
2023-07-13 11:55:00
['20230404', '20230405', '20230406', '20230407', '20230408', '20230409', '20230410', '20230411', '20230412', '20230413', '20230414', '20230415', '20230416', '20230417', '20230418', '20230419', '20230420', '20230421', '20230422', '20230423', '20230424', '20230425', '20230426', '20230427', '20230428', '20230429', '20230430', '20230501', '20230502', '20230503', '20230504', '20230505', '20230506', '20230507', '20230508', '20230509', '20230510', '20230511', '20230512', '20230513', '20230514', '20230515', '20230516', '20230517', '20230518', '20230519', '20230520', '20230521', '20230522', '20230523', '20230524', '20230525', '20230526', '20230527', '20230528', '20230529', '20230530', '20230531', '20230601', '20230602', '20230603', '20230604', '20230605', '20230606', '20230607', '20230608', '20230609', '20230610', '20230611', '20230612', '20230613', '20230614', '2023

ValueError: read length must be non-negative or -1

In [ ]:

t1all = time.time()

print(' ')
print(' ')
print(' ')
print('------------------')
print('Runtime for full data update web code:', np.round((t1all-t0all)/60,2), 'minutes')
print('--------------------------------------------------------------------------------------')



In [ ]:
sys.exit()

### STEREO-A science data

In [11]:
print(' ')
print('------ download STEREO-A science data ')
get_stereoa_sci=1


if get_stereoa_sci >0: hd.download_stereoa_science_merged()

In [58]:

sys.exit()

In [ ]:




#on linux
#print('download new Wind data files without overwriting existing files')

#on mac for testing
#wind_data_path='/Users/chris/python/data/wind/wind_mfi_k0'
#os.system('curl -nc --directory-prefix='+wind_data_path+' "ftps://spdf.gsfc.nasa.gov/pub/data/wind/mfi/mfi_k0/2020/*.cdf"')



#wind_data_path='/perm/aswo/data/wind/wind_mfi_k0'
#print(wind_data_path)
#os.system('wget -nc --directory-prefix='+wind_data_path+' "ftps://spdf.gsfc.nasa.gov/pub/data/wind/mfi/mfi_k0/2020/*.cdf"')
#wind_data_path='/nas/helio/data/heliosat/data/wind_swe_h1'
#os.system('wget -nc --directory-prefix='+wind_data_path+' "ftps://spdf.gsfc.nasa.gov/pub/data/wind/swe/swe_h1/2020/*.cdf"')

filewin="wind_2018_now_gse.p" 
start=datetime.datetime(2018, 1, 1)
end=datetime.datetime.utcnow()
if get_new_data: hd.save_wind_data(data_path,filewin,start,end,heeq=False)
[win,hwin]=pickle.load(open(data_path+filewin, "rb" ) )  

filewin="wind_2018_now_heeq.p" 
start=datetime.datetime(2018, 1, 1)
end=datetime.datetime.utcnow()
if get_new_data: hd.save_wind_data(data_path,filewin,start,end,heeq=True)

start=win.time[-1]-datetime.timedelta(days=100)
end=datetime.datetime.utcnow()         
hp.plot_insitu_update(win, start, end,'Wind',plot_path,now=True)

In [ ]:
sys.exit()

In [ ]:


############### write header file for daily updates
text = open('/nas/helio/data/insitu_python/data_update_headers.txt', 'w')
text.write('Contains headers for the data files which are updated in real time.'+'\n \n')
text.write('File creation date:  '+datetime.datetime.utcnow().strftime("%Y-%b-%d %H:%M") +' \n \n')


text.write('NOAA real time solar wind: '+filenoaa+'\n \n'+ hnoaa+' \n \n')
text.write('load with: >> [noaa,hnoaa]=pickle.load(open("'+data_path+filenoaa+'", "rb"))') 
text.write(' \n \n \n \n')

text.write('STEREO-A beacon: '+filesta_sceq+'\n \n'+ hsta+' \n \n')
text.write('load with: >> [sta,hsta]=pickle.load(open("'+data_path+filesta+'", "rb"))') 
text.write(' \n \n \n \n')

text.write('Wind: '+filewin+'\n \n'+ hwin+' \n \n')
text.write('load with: >> [win,hwin]=pickle.load(open("'+data_path+filewin+'", "rb" ))') 
text.write(' \n \n \n \n')


text.write('OMNI2: '+fileomni+'\n \n'+ ho+' \n \n')
text.write('load with: >> [o,ho]=pickle.load(open("'+data_path+fileomni+'", "rb" ))') 
text.write(' \n \n \n \n')

text.close()

In [4]:
#for easier debugging - do not download and process data but do everything else
#get_new_data=True


#~/miniconda/envs/helio/bin/python /home/cmoestl/pycode/heliocats/sc_positions_insitu_orbit1.py
#~/miniconda/envs/helio/bin/python /home/cmoestl/pycode/heliocats/sc_positions_insitu_orbit2.py


################################# PSP data update

################################## USE THIS ################################
# old
# load PSP data from server on linux command line onto leo server
# go to heliosat directory /nas/helio/data/heliosat/data/psp_fields_l2
# wget -nc "ftps://spdf.gsfc.nasa.gov/pub/data/psp/fields/l2/mag_rtn_1min/2019/*.cdf"
# wget -nc "ftps://spdf.gsfc.nasa.gov/pub/data/psp/fields/l2/mag_rtn_1min/2020/*.cdf"
#  /nas/helio/data/heliosat/data/psp_spc_l3
# wget -nc "ftps://spdf.gsfc.nasa.gov/pub/data/psp/sweap/spc/l3/l3i/2019/*.cdf"
# wget -nc "ftps://spdf.gsfc.nasa.gov/pub/data/psp/sweap/spc/l3/l3i/2020/*.cdf"
#new
#use beginning of icmecat.ipynb
############################################################################

# print('load PSP data') #from heliosat, converted to SCEQ similar to STEREO-A/B
#set time there
#change date in hd.save_psp_data
#filepsp='psp_2018_2020_nov_rtn.p'
#hd.save_psp_data(data_path,filepsp, sceq=False)   

#filepsp='psp_2018_2020_sceq.p'
#hd.save_psp_data(data_path,filepsp, sceq=True)   



############################################ STEREO - A science data update



-----------------------------
data files:

sftp.put(path+'Wind_now.png')  # upload file to public/ on remote
sftp.put(path+'STEREO-A_beacon_14_days_now.png')
sftp.put(path+'OMNI2_now.png')


sftp.put(path+'NOAA_RTSW_PREDSTORM_55days_now.png')
sftp.put(path+'NOAA_RTSW_PREDSTORM_14days_now.png')
sftp.put(path+'NOAA_RTSW_PREDSTORM_3days_now.png')
sftp.put(path+'OMNI2_and_NOAA_RTSW_now.png')


----------------------------

In [ ]:
################## NOAA real time
################ load jsons

print('download NOAA real time solar wind plasma and mag')
datestr=str(datetime.datetime.utcnow().strftime("%Y_%b_%d_%H_%M"))
print(datestr+' UTC')

plasma='http://services.swpc.noaa.gov/products/solar-wind/plasma-7-day.json'
mag='http://services.swpc.noaa.gov/products/solar-wind/mag-7-day.json'

try: urllib.request.urlretrieve(plasma, noaa_path+'plasma-7-day_'+datestr+'.json')
except urllib.error.URLError as e:
  print(' ', plasma,' ',e.reason)

try: urllib.request.urlretrieve(mag, noaa_path+'mag-7-day_'+datestr+'.json')
except urllib.error.URLError as e:
  print(' ', mag,' ',e.reason)
  
print()


# my version:
#filenoaa='noaa_rtsw_jan_2020_now.p'
#if get_new_data: hd.save_noaa_rtsw_data(data_path,noaa_path,filenoaa)
#[noaa,hnoaa]=pickle.load(open(data_path+filenoaa, "rb" ) ) 

#################### with Rachel Bailey's data from predstorm
noaapath='/nas/helio/realcode/real/predstorm/data/rtsw_min_last100days.h5'
filenoaa='noaa_rtsw_last100days_now.p'
if get_new_data: hd.save_noaa_rtsw_data_predstorm(data_path,noaapath,filenoaa)        
[noaa,hnoaa]=pickle.load(open(data_path+filenoaa, "rb" ) ) 
    
              
############### make 3 plots

#for ICMEs
start=noaa.time[-1]-datetime.timedelta(days=3)
end=datetime.datetime.utcnow() #noaa.time[-1]     
hp.plot_insitu_update(noaa, start, end,'NOAA_RTSW_PREDSTORM_3days',plot_path,now=True)

start=noaa.time[-1]-datetime.timedelta(days=14)
end=datetime.datetime.utcnow() #noaa.time[-1]     
hp.plot_insitu_update(noaa, start, end,'NOAA_RTSW_PREDSTORM_14days',plot_path,now=True)

start=noaa.time[-1]-datetime.timedelta(days=55)
end=datetime.datetime.utcnow() #noaa.time[-1]     
hp.plot_insitu_update(noaa, start, end,'NOAA_RTSW_PREDSTORM_55days',plot_path,now=True)

print()

#make quick new plot for ICMEs
# hd.save_noaa_rtsw_data(data_path,noaa_path,filenoaa)
# start=noaa.time[-1]-datetime.timedelta(days=3)
# end=datetime.datetime.utcnow() #noaa.time[-1]     
# hp.plot_insitu(noaa, start, end,'NOAA_RTSW','/home/cmoestl/pycode/heliocats')





####################################### OMNI2
fileomni="omni_1963_now.p"
if get_new_data: hd.save_omni_data(data_path,fileomni)
[o,ho]=pickle.load(open(data_path+fileomni, "rb" ) )  

start=datetime.datetime.utcnow() - datetime.timedelta(days=365)
end=datetime.datetime.utcnow() 
hp.plot_insitu_update(o, start, end,'OMNI2',plot_path,now=True)


########################## add NOAA RTSW to OMNI data and make combined plot

#get index of last OMNI data
last_omni_index=np.where(np.isfinite(o.bt) == True)[0][-1]
#get time for this index
last_omni_time=o.time[last_omni_index]
#add utc timezone awareness
last_omni_time_utc=last_omni_time.astimezone(tz=datetime.timezone.utc)
#get index where omni ends in noaa data
noaa_omni_end=np.where(noaa.time>last_omni_time_utc)[0][0]


#length of NOAA data
size_noaa=np.size(noaa)-noaa_omni_end

combi_omni_noaa=np.zeros(last_omni_index+size_noaa,dtype=[('time',object),('bx', float),('by', float),\
                ('bz', float),('bt', float),('vt', float),('np', float),('tp', float),\
                ('x', float),('y', float),('z', float),\
                ('r', float),('lat', float),('lon', float)])   

#convert to recarray
combi_omni_noaa = combi_omni_noaa.view(np.recarray)  
#stack omni and noaa data
combi_omni_noaa.time=np.hstack((o.time[0:last_omni_index],noaa.time[noaa_omni_end-1:-1]))
combi_omni_noaa.bx=np.hstack((o.bx[0:last_omni_index],noaa.bx[noaa_omni_end-1:-1]))
combi_omni_noaa.by=np.hstack((o.by[0:last_omni_index],noaa.by[noaa_omni_end-1:-1]))
combi_omni_noaa.bz=np.hstack((o.bz[0:last_omni_index],noaa.bz[noaa_omni_end-1:-1]))
combi_omni_noaa.bt=np.hstack((o.bt[0:last_omni_index],noaa.bt[noaa_omni_end-1:-1]))
combi_omni_noaa.vt=np.hstack((o.vt[0:last_omni_index],noaa.vt[noaa_omni_end-1:-1]))
combi_omni_noaa.np=np.hstack((o.np[0:last_omni_index],noaa.np[noaa_omni_end-1:-1]))
combi_omni_noaa.tp=np.hstack((o.tp[0:last_omni_index],noaa.tp[noaa_omni_end-1:-1]))

print('Merging NOAA OMNI done')

start=datetime.datetime.utcnow() -datetime.timedelta(days=365)
end=datetime.datetime.utcnow() 
hp.plot_insitu_update(combi_omni_noaa, start, end,'OMNI2_and_NOAA_RTSW',plot_path,now=True)




#for making quick plots
# plot_path='/home/cmoestl/pycode/heliocats/'
# fileomni="omni_1963_now.p"
# [o,ho]=pickle.load(open(data_path+fileomni, "rb" ) )  
# start=datetime.datetime.utcnow() -datetime.timedelta(days=365)
# end=datetime.datetime.utcnow() 
# hp.plot_insitu_update(o, start, end,'OMNI2',plot_path,now=True)





################################### STEREO-A

from heliocats import plot as hp
importlib.reload(hp) #reload again while debugging



#start=datetime.datetime(2020, 8, 1)
#end=datetime.datetime.utcnow()
#filesta_sceq="stereoa_2020_august_now_sceq_beacon.p" 
#filesta_rtn="stereoa_2020_august_now_rtn_beacon.p" 

#if get_new_data: 
#    hd.save_stereoa_beacon_data(data_path,filesta_sceq,start,end,sceq=True)
#    hd.save_stereoa_beacon_data(data_path,filesta_rtn,start,end,sceq=False)


start=datetime.datetime(2020, 1, 1)
end=datetime.datetime.utcnow()
filesta_sceq="stereoa_2020_now_sceq_beacon.p" 
filesta_rtn="stereoa_2020_now_rtn_beacon.p" 

if get_new_data: 
    hd.save_stereoa_beacon_data(data_path,filesta_sceq,start,end,sceq=True)
    hd.save_stereoa_beacon_data(data_path,filesta_rtn,start,end,sceq=False)

[sta,hsta]=pickle.load(open(data_path+filesta_rtn, "rb" ) ) 

start=sta.time[-1]-datetime.timedelta(days=14)
end=datetime.datetime.utcnow()     
hp.plot_insitu_update_stereoa_beacon(sta, start, end,'STEREO-A_beacon_14_days',plot_path,now=True)




####################################### Wind

print('download Wind 2020 files without overwriting existing files into ',data_path)
wind_data_path='/nas/helio/data/heliosat/data/wind_mfi_k0'
os.system('wget -nc --directory-prefix='+wind_data_path+' "ftps://spdf.gsfc.nasa.gov/pub/data/wind/mfi/mfi_k0/2020/*.cdf"')
wind_data_path='/nas/helio/data/heliosat/data/wind_swe_h1'
os.system('wget -nc --directory-prefix='+wind_data_path+' "ftps://spdf.gsfc.nasa.gov/pub/data/wind/swe/swe_h1/2020/*.cdf"')


filewin="wind_2018_now_gse.p" 
start=datetime.datetime(2018, 1, 1)
end=datetime.datetime.utcnow()
if get_new_data: hd.save_wind_data(data_path,filewin,start,end,heeq=False)
[win,hwin]=pickle.load(open(data_path+filewin, "rb" ) )  

filewin="wind_2018_now_heeq.p" 
start=datetime.datetime(2018, 1, 1)
end=datetime.datetime.utcnow()
if get_new_data: hd.save_wind_data(data_path,filewin,start,end,heeq=True)

start=win.time[-1]-datetime.timedelta(days=100)
end=datetime.datetime.utcnow()         
hp.plot_insitu_update(win, start, end,'Wind',plot_path,now=True)



############### write header file for daily updates
text = open('/nas/helio/data/insitu_python/data_update_headers.txt', 'w')
text.write('Contains headers for the data files which are updated in real time.'+'\n \n')
text.write('File creation date:  '+datetime.datetime.utcnow().strftime("%Y-%b-%d %H:%M") +' \n \n')


text.write('NOAA real time solar wind: '+filenoaa+'\n \n'+ hnoaa+' \n \n')
text.write('load with: >> [noaa,hnoaa]=pickle.load(open("'+data_path+filenoaa+'", "rb"))') 
text.write(' \n \n \n \n')

text.write('STEREO-A beacon: '+filesta_sceq+'\n \n'+ hsta+' \n \n')
text.write('load with: >> [sta,hsta]=pickle.load(open("'+data_path+filesta+'", "rb"))') 
text.write(' \n \n \n \n')

text.write('Wind: '+filewin+'\n \n'+ hwin+' \n \n')
text.write('load with: >> [win,hwin]=pickle.load(open("'+data_path+filewin+'", "rb" ))') 
text.write(' \n \n \n \n')


text.write('OMNI2: '+fileomni+'\n \n'+ ho+' \n \n')
text.write('load with: >> [o,ho]=pickle.load(open("'+data_path+fileomni+'", "rb" ))') 
text.write(' \n \n \n \n')

text.close()